# VaR and Portfolio Risk Analysis

Selected stocks: AAPL, META, NVDA, TSLA

In [127]:
import pandas as pd
import numpy as np
from scipy import stats

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 6)

In [128]:
# Load data
df = pd.read_excel('data/spx_returns_weekly.xlsx', sheet_name='s&p500 rets')
stocks = ['AAPL', 'META', 'NVDA', 'TSLA']
returns = df[stocks]
print(f"Data shape: {returns.shape}")

Data shape: (542, 4)


In [129]:
# 1.1 Individual stock risk metrics
def calculate_cvar(returns, alpha=0.05):
    var = returns.quantile(alpha)
    return returns[returns <= var].mean()

results = pd.DataFrame(index=stocks, columns=['Volatility', 'VaR (0.05)', 'CVaR (0.05)'])
for stock in stocks:
    results.loc[stock] = [
        returns[stock].std(),
        returns[stock].quantile(0.05),
        calculate_cvar(returns[stock])
    ]

print("\n1.1 Individual Stock Risk Metrics (%):\n")
print((results * 100).round(4))


1.1 Individual Stock Risk Metrics (%):

     Volatility VaR (0.05) CVaR (0.05)
AAPL   3.836155   -5.63663   -8.312492
META   4.872151  -7.001167  -10.319606
NVDA   6.424578  -8.685282  -11.645514
TSLA   8.132338  -11.73973   -14.78137


## 1.2 Equally-Weighted Portfolio

In [130]:
# Create portfolio
weights = np.array([0.25, 0.25, 0.25, 0.25])
portfolio_returns = (returns * weights).sum(axis=1)

# Calculate metrics
portfolio_volatility = portfolio_returns.std()
portfolio_var = portfolio_returns.quantile(0.05)
portfolio_cvar = calculate_cvar(portfolio_returns)

comparison = results.copy()
comparison.loc['Portfolio (EW)'] = [portfolio_volatility, portfolio_var, portfolio_cvar]

print("\n1.2 Portfolio vs Individual Stocks (%):\n")
print((comparison * 100).round(4))

avg_vol = results['Volatility'].mean()
print(f"\nVolatility reduction: {(avg_vol - portfolio_volatility)/avg_vol*100:.2f}%")


1.2 Portfolio vs Individual Stocks (%):

               Volatility VaR (0.05) CVaR (0.05)
AAPL             3.836155   -5.63663   -8.312492
META             4.872151  -7.001167  -10.319606
NVDA             6.424578  -8.685282  -11.645514
TSLA             8.132338  -11.73973   -14.78137
Portfolio (EW)   4.375791   -6.19499   -8.499232

Volatility reduction: 24.77%


**Analysis:** Portfolio volatility (4.38%) is 24.77% lower than average individual volatility. Driven by imperfect correlation.

## 1.3 Portfolio Without Most Volatile Asset

In [131]:
# Remove most volatile
most_volatile = results['Volatility'].idxmax()
most_volatile_idx = stocks.index(most_volatile)
weights_reduced = weights.copy()
weights_reduced[most_volatile_idx] = 0

portfolio_returns_reduced = (returns * weights_reduced).sum(axis=1)
portfolio_reduced_vol = portfolio_returns_reduced.std()
portfolio_reduced_var = portfolio_returns_reduced.quantile(0.05)
portfolio_reduced_cvar = calculate_cvar(portfolio_returns_reduced)

comparison_full = comparison.copy()
comparison_full.loc[f'Portfolio (excl. {most_volatile})'] = [
    portfolio_reduced_vol, portfolio_reduced_var, portfolio_reduced_cvar
]

print(f"\n1.3 Impact of Removing {most_volatile} (%):\n")
print((comparison_full * 100).round(4))

vol_diff = portfolio_volatility - portfolio_reduced_vol
print(f"\nVolatility reduction: {vol_diff*100:.2f}pp ({vol_diff/portfolio_volatility*100:.2f}%)")


1.3 Impact of Removing TSLA (%):

                       Volatility VaR (0.05) CVaR (0.05)
AAPL                     3.836155   -5.63663   -8.312492
META                     4.872151  -7.001167  -10.319606
NVDA                     6.424578  -8.685282  -11.645514
TSLA                     8.132338  -11.73973   -14.78137
Portfolio (EW)           4.375791   -6.19499   -8.499232
Portfolio (excl. TSLA)   3.028175  -4.246251   -6.053113

Volatility reduction: 1.35pp (30.80%)


**Analysis:** TSLA's marginal contribution (1.35pp) is only 67% of naive calculation (25% × 8.13% = 2.03pp) due to diversification effect.

# 2. Dynamic Measures

## 2.1 Conditional Statistics

In [132]:
# Rolling volatility
window_size = 26
rolling_volatility = portfolio_returns.rolling(window=window_size).std()
conditional_volatility = rolling_volatility.iloc[-1]

# Normal VaR/CVaR
z_005 = -1.65
q = 0.05
normal_var = z_005 * conditional_volatility
phi_z = stats.norm.pdf(z_005)
normal_cvar = -(phi_z / q) * conditional_volatility

weeks_per_year = 52
summary_2_1 = pd.DataFrame({
    'Metric': ['Volatility (Weekly)', 'Volatility (Annualized)', 'VaR (0.05)', 'CVaR (0.05)'],
    'Conditional (2.1)': [
        conditional_volatility,
        conditional_volatility * np.sqrt(weeks_per_year),
        normal_var,
        normal_cvar
    ],
    'Unconditional (1.2)': [
        portfolio_volatility,
        portfolio_volatility * np.sqrt(weeks_per_year),
        portfolio_var,
        portfolio_cvar
    ]
})

print("\n2.1 Conditional vs Unconditional (%):\n")
print((summary_2_1.set_index('Metric') * 100).round(4))


2.1 Conditional vs Unconditional (%):

                         Conditional (2.1)  Unconditional (1.2)
Metric                                                         
Volatility (Weekly)                 5.4049               4.3758
Volatility (Annualized)            38.9750              31.5543
VaR (0.05)                         -8.9180              -6.1950
CVaR (0.05)                       -11.0546              -8.4992


**Analysis:** Conditional vol (5.40%) > Unconditional (4.38%) → recent 26 weeks more volatile than historical average.

## 2.2 VaR Backtesting

In [133]:
# Expanding and rolling VaR
expanding_volatility = portfolio_returns.expanding(min_periods=window_size).std()
var_expanding = z_005 * expanding_volatility.shift(1)
var_rolling = z_005 * rolling_volatility.shift(1)

# Hit test
hits_expanding = portfolio_returns < var_expanding
hits_rolling = portfolio_returns < var_rolling

num_hits_exp = hits_expanding.sum()
num_valid_exp = var_expanding.notna().sum()
hit_rate_exp = num_hits_exp / num_valid_exp * 100

num_hits_roll = hits_rolling.sum()
num_valid_roll = var_rolling.notna().sum()
hit_rate_roll = num_hits_roll / num_valid_roll * 100

backtest_summary = pd.DataFrame({
    'Method': ['Expanding', 'Rolling (26w)'],
    'Observations': [num_valid_exp, num_valid_roll],
    'Hits': [num_hits_exp, num_hits_roll],
    'Hit Rate (%)': [hit_rate_exp, hit_rate_roll],
    'Expected (%)': [5.0, 5.0]
})

print("\n2.2 Backtesting Results:\n")
print(backtest_summary.to_string(index=False))


2.2 Backtesting Results:

       Method  Observations  Hits  Hit Rate (%)  Expected (%)
    Expanding           516    27      5.232558           5.0
Rolling (26w)           516    22      4.263566           5.0


**Analysis:** Hit rate ≈ 5% indicates well-calibrated model. Hit rate > 5% means underestimating risk; < 5% means overestimating.